# Post-processing techniques

In [34]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [23]:
df = pd.read_csv('../../data/final_features_df.csv')
df.head()

,Unnamed: 0,Age,Income,faves_pca0,faves_pca1,unfaves_pca0,unfaves_pca1,accessories,alcohol,animamted,...,Drama.2,Entertainment (Variety Shows),Factual,Learning,Music,News,Religion &amp; Ethics,Sport.1,Weather,Rating_bin
0,0,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,1,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,2,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,3,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,4,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [24]:
df = df.fillna(0)

## Data preparation

In [25]:
X = df.drop(columns='Rating_bin')
y = df['Rating_bin']

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

## Baseline model: DecisionTree

In [27]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred)

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      7775
           1       0.40      0.39      0.39      1255

    accuracy                           0.83      9030
   macro avg       0.65      0.65      0.65      9030
weighted avg       0.83      0.83      0.83      9030



array([[7048,  727],
       [ 769,  486]])

## Métrica de fairness: average odds

In [28]:
import numpy as np

def statistical_parity(y, y_, Z, priv=None):
  if priv is None:
    values = np.unique(Z)
    counts = [np.mean(y[Z==z]) for z in values]
    priv = values[np.argmax(counts)]
    unpriv = [z for z in values if z != priv]
    print('Automatic priviledged value is', priv)
  else:
    unpriv = [z for z in values if z != priv]
  
  return np.array([np.mean([y_i for y_i, zi in zip(y_, Z) if zi == unp]) - np.mean([y_i for y_i, zi in zip(y_, Z) if zi == priv])
                   for unp in unpriv])

In [43]:
Z_train = X_train['Gender_M']==1
Z_val = X_val['Gender_M']==1

## Desempenho do baseline model

In [42]:
y_val_ = clf.predict(X_val)

print('F1-score:', f1_score(y_val, y_val_))
print('Statistical parity', statistical_parity(y_val, y_val_, Z_val))

F1-score: 0.393841166936791
Automatic priviledged value is True
Statistical parity [-0.02351367]


## Estratégia 1: Threshold Optimizer

In [55]:
from fairlearn.postprocessing import ThresholdOptimizer

postprocess_est = ThresholdOptimizer(
                   estimator=clf,
                   constraints="demographic_parity",
                   objective="true_positive_rate",
                   prefit=True,
                   predict_method='predict_proba')
postprocess_est.fit(X_train, y_train, sensitive_features=Z_train)

y_val_ = postprocess_est.predict(X_val, sensitive_features=Z_val)

print('F1-score:', f1_score(y_val, y_val_))
print('Statistical parity', statistical_parity(y_val, y_val_, Z_val))

F1-score: 0.382341650671785
Automatic priviledged value is True
Statistical parity [-0.00048215]


### Avaliando hiperparâmetros

O ThresholdOptimizer possui dois hiperparâmetros que geram impacto direto nas métricas, são eles: `constraints` e `objective`. Avaliando os resultados:

In [69]:
possible_constraints = [
    'demographic_parity',
    'selection_rate_parity',
    'equalized_odds',
    'true_positive_rate_parity',
    'true_negative_rate_parity',
    'false_positive_rate_parity',
    'false_negative_rate_parity',
]
possible_objectives = [
    'accuracy_score',
    'balanced_accuracy_score',
    'selection_rate',
    'true_positive_rate',
    'true_negative_rate',
]

impossible_pairs = [
  ['equalized_odds', 'selection_rate'],
  ['equalized_odds', 'true_positive_rate'],
  ['equalized_odds', 'true_negative_rate'],
]

results = dict()

y_val_ = clf.predict(X_val)
baseline_f1 = f1_score(y_val, y_val_)
baseline_sp = statistical_parity(y_val, y_val_, Z_val)[0]
print(f'Baseline F1: {baseline_f1}')
print(f'Baseline statistical parity: {baseline_sp}')

results['baseline']= [baseline_f1, baseline_sp]

for constraint in possible_constraints:
    for objective in possible_objectives:
        if [constraint, objective] not in impossible_pairs:
            print('---')
            print(f'Optimizing using objective "{objective}" and constraint "{constraint}"')
            postprocess_est = ThresholdOptimizer(
                   estimator=clf,
                   constraints="demographic_parity",
                   objective="true_positive_rate",
                   prefit=True,
                   predict_method='predict_proba')
            postprocess_est.fit(X_train, y_train, sensitive_features=Z_train)
            
            y_val_ = postprocess_est.predict(X_val, sensitive_features=Z_val)
            alt_f1 = f1_score(y_val, y_val_)
            alt_sp = statistical_parity(y_val, y_val_, Z_val)[0]
            print(f'F1-score: {alt_f1}')
            print(f'Statistical parity: {alt_sp}')
            results[f'{objective}__{constraint}']= [alt_f1, alt_sp]

Automatic priviledged value is True
Baseline F1: 0.393841166936791
Baseline statistical parity: -0.023513669190545816
---
Optimizing using objective "accuracy_score" and constraint "demographic_parity"
Automatic priviledged value is True
F1-score: 0.39148936170212767
Statistical parity: -0.003431279901153439
---
Optimizing using objective "balanced_accuracy_score" and constraint "demographic_parity"
Automatic priviledged value is True
F1-score: 0.38396299151888974
Statistical parity: -0.0018864807250463245
---
Optimizing using objective "selection_rate" and constraint "demographic_parity"
Automatic priviledged value is True
F1-score: 0.38255547054322875
Statistical parity: 0.0015464063329694577
---
Optimizing using objective "true_positive_rate" and constraint "demographic_parity"
Automatic priviledged value is True
F1-score: 0.3907514450867052
Statistical parity: -0.0021985905877179857
---
Optimizing using objective "true_negative_rate" and constraint "demographic_parity"
Automatic pr

In [70]:
results

{'baseline': [0.393841166936791, -0.023513669190545816],
 'accuracy_score__demographic_parity': [0.39148936170212767,
  -0.003431279901153439],
 'balanced_accuracy_score__demographic_parity': [0.38396299151888974,
  -0.0018864807250463245],
 'selection_rate__demographic_parity': [0.38255547054322875,
  0.0015464063329694577],
 'true_positive_rate__demographic_parity': [0.3907514450867052,
  -0.0021985905877179857],
 'true_negative_rate__demographic_parity': [0.38977140643161556,
  -0.004601611528329025],
 'accuracy_score__selection_rate_parity': [0.38036335523772713,
  -0.003087991195351858],
 'balanced_accuracy_score__selection_rate_parity': [0.3829457364341085,
  -0.004289501665657364],
 'selection_rate__selection_rate_parity': [0.3870220162224797,
  -0.002744702489550277],
 'true_positive_rate__selection_rate_parity': [0.3810623556581986,
  -0.0011999033134431625],
 'true_negative_rate__selection_rate_parity': [0.3866256725595696,
  -0.0005133259018400005],
 'accuracy_score__equaliz

In [75]:
best_combination = 'baseline'
best_combination_f1 = results[best_combination][0]
best_combination_sp = results[best_combination][1]

for combination, (alt_f1, alt_sp) in results.items():
    if best_combination_sp < alt_sp:
        best_combination = combination
        best_combination_f1 = alt_f1
        best_combination_sp = alt_sp

objective, constraint = best_combination.split('__')
print(f'Best objetive: {objective}')
print(f'Best constraint: {constraint}')
print(f'F1-score: {best_combination_f1}')
print(f'Statistical parity: {best_combination_sp}')

Best objetive: selection_rate
Best constraint: demographic_parity
F1-score: 0.38255547054322875
Statistical parity: 0.0015464063329694577
